In [1]:
import json

def print_json(filename: str):
    with open(filename, 'r') as openfile:
        json_object = json.load(openfile)

    json_object = json.dumps(json_object, indent=2)
    print(json_object)

### Introduction: Simple example

In [2]:
from rtb_argparse.config import ArgparseConfig, json_config_parser

parser =  ArgparseConfig(fromfile_prefix_chars="@", file_parser=json_config_parser)
parser.add_argument('arg1', type=str)
parser.add_argument("--foo", type=str)
parser.add_argument("--name", type=str)

parser.print_help()

usage: ipykernel_launcher.py [-h] [--foo FOO] [--name NAME] arg1

positional arguments:
  arg1

options:
  -h, --help   show this help message and exit
  --foo FOO
  --name NAME


In [3]:
filename = "data/introduction_simple_example.json"
print_json(filename)

[
  "text",
  {
    "--foo": "foo",
    "--name": "myname"
  }
]


In [4]:
opt = parser.parse_args([f"@{filename}"])
print(opt.arg1, opt.foo, opt.name)

text foo myname


In [5]:
filename = "data/introduction_complex_example.json"
print_json(filename)

[
  "text",
  {
    "@default": {
      "--foo": "foo",
      "--name": "myname"
    },
    "@conf1": {
      "--foo": "oof",
      "--name": "conf1"
    }
  }
]


In [6]:
opt = parser.parse_args([f"@{filename}"])
print(opt.arg1, opt.foo, opt.name)

text foo myname


In [7]:
opt = parser.parse_args([f"@{filename}@conf1"])
print(opt.arg1, opt.foo, opt.name)

text oof conf1
